In [ ]:
import sys
sys.path.append('../')

In [ ]:
import os
import pycolmap
from pathlib import Path
from collections import defaultdict
import h5py as h5

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from hloc.utils.io import list_h5_names, get_matches, get_keypoints, find_pair
from hloc.visualization import plot_images, plot_keypoints, plot_matches, read_image, add_text, cm_RdGn

In [ ]:
DIR = "../image-matching-challenge-2023"
MODE = "train"
NAME1 = "SP+LG-rot"
NAME2 = "sift+NN"
dataset = "heritage" # "heritage", "haiper", "urban"
scene = "dioscuri" # "dioscuri", "cyprus", "wall", "kyiv-puppet-theater", "bike", "chairs", "fountain"

In [ ]:
features1_dir = f"../outputs/{NAME1}/{dataset}/{scene}/features.h5"
features2_dir = f"../outputs/{NAME2}/{dataset}/{scene}/features.h5"

matches1_dir = f"../outputs/{NAME1}/{dataset}/{scene}/matches.h5"
matches2_dir = f"../outputs/{NAME2}/{dataset}/{scene}/matches.h5"

images = Path(os.path.join(DIR, MODE, dataset, scene, "images"))

In [ ]:
features1 = h5.File(features1_dir, "r")
features2 = h5.File(features2_dir, "r")

matches1_path = h5.File(matches1_dir, "r")
matches2_path = h5.File(matches2_dir, "r")

In [ ]:
image_names = os.listdir(images)

# Concatenate Features

In [ ]:
def concat_features(features1: Path, features2: Path, out_path: Path):
    # read features
    with h5.File(features1, "r") as f1:
        with h5.File(features2, "r") as f2:
            ensemble_features = {
                img: {
                    "keypoints": np.concatenate(
                        [f1[img]["keypoints"], f2[img]["keypoints"]], axis=0,
                    ),
                    "scores": np.concatenate(
                        [f1[img]["scores"], f2[img]["scores"]], axis=0
                    ),
                    "feats1": f1[img]["keypoints"].shape[0],
                    "feats2": f2[img]["keypoints"].shape[0],
                }
                for img in tqdm(f1.keys(), desc="concatenating features", ncols=80)
            }

    # write features
    ens_kp_ds = h5.File(out_path, "w")
    for img in ensemble_features:
        ens_kp_ds.create_group(img)
        for k in ensemble_features[img].keys():
            ens_kp_ds[img].create_dataset(k, data=ensemble_features[img][k])

    ens_kp_ds.close()

In [ ]:
ens_feat_path = Path(f"../outputs/ensemble/{dataset}/{scene}/ens_features.h5")

if not ens_feat_path.exists():
    ens_feat_path.parent.mkdir(parents=True, exist_ok=True)

concat_features(features1_dir, features2_dir, ens_feat_path)

In [ ]:
# Features 1
plot_images([read_image(images / imname) for imname in image_names[:4]])
plot_keypoints([get_keypoints(features1_dir, imname) for imname in image_names[:4]])

In [ ]:
# Features 2
plot_images([read_image(images / imname) for imname in image_names[:4]])
plot_keypoints([get_keypoints(features2_dir, imname) for imname in image_names[:4]])

In [ ]:
# Ensemble Features
plot_images([read_image(images / imname) for imname in image_names[:4]])
plot_keypoints([get_keypoints(ens_feat_path, imname) for imname in image_names[:4]])

# Concatenate Matches

In [ ]:
def reverse_matches(matches, scores, nkps1, nkps2):
    rev_matches = np.ones(nkps2) * -1
    rev_scores = np.zeros(nkps2)

    assert len(matches) == nkps1, "Number of matches must equal number of keypoints in image 1"
    assert np.max(matches) < nkps2, "Matches must be indices of keypoints in image 2"

    # matches is a list of length nkps1 with each value being either -1 or the index of the match in nkps2
    for i, m in enumerate(matches):
        if m != -1:
            rev_matches[m] = i
            rev_scores[m] = scores[i]

    return rev_matches.astype(int), rev_scores

In [ ]:
def concat_matches(matches1_path: Path, matches2_path: Path, ensemble_features_path: Path, out_path: Path):
    # concat matches
    ensemble_matches = {}
    with h5.File(matches1_path, "r") as matches1:
        with h5.File(matches2_path, "r") as matches2:
            with h5.File(ensemble_features_path, "r") as ensemble_features:
                pairs = sorted(list_h5_names(matches1_path))
                for pair in tqdm(pairs, desc="concatenating matches", ncols=80):
                    name0, name1 = pair.split("/")

                    # prepare dict
                    if name0 not in ensemble_matches:
                        ensemble_matches[name0] = {}
                    if name1 not in ensemble_matches[name0]:
                        ensemble_matches[name0][name1] = {}

                    # get matches1
                    p1, rev1 = find_pair(matches1, name0, name1)
                    m1 = matches1[p1]["matches0"].__array__()
                    sc1 = matches1[p1]["matching_scores0"].__array__()

                    if rev1:
                        m1, sc1 = reverse_matches(m1, sc1, ensemble_features[name1]["feats1"], ensemble_features[name0]["feats1"])


                    # get matches2
                    p2, rev2 = find_pair(matches2, name0, name1)
                    m2 = matches2[p2]["matches0"].__array__()
                    sc2 = matches2[p2]["matching_scores0"].__array__()

                    if rev2:
                        m2, sc2 = reverse_matches(m2, sc2, ensemble_features[name1]["feats2"], ensemble_features[name0]["feats2"])


                    # concat matches
                    offset = ensemble_features[name0]["feats1"]
                    m2 += offset * np.where(m2 != -1, 1, 0)

                    ensemble_matches[name0][name1]["matches0"] = np.concatenate(
                        [m1, m2],
                        axis=0,
                    )

                    ensemble_matches[name0][name1]["matching_scores0"] = np.concatenate(
                        [sc1, sc2],
                        axis=0,
                    )

    ens_matches_ds = h5.File(out_path, "w")
    for img1 in ensemble_matches:
        ens_matches_ds.create_group(img1)
        for img2 in ensemble_matches[img1].keys():
            ens_matches_ds[img1].create_group(img2)
            for k in ensemble_matches[img1][img2].keys():
                ens_matches_ds[img1][img2].create_dataset(
                    k, data=ensemble_matches[img1][img2][k]
                )

    ens_matches_ds.close()

In [ ]:
ens_matches_path = Path(f"../outputs/ensemble/{dataset}/{scene}/ens_matches.h5")
concat_matches(matches1_dir, matches2_dir, ens_feat_path, ens_matches_path)

In [ ]:
pairs = sorted(list_h5_names(ens_matches_path))
len(pairs)

In [ ]:
pairs = sorted(list_h5_names(matches1_dir))
match_matrix1 = -np.ones([len(image_names), len(image_names)])
for pair in pairs:
    name0, name1 = pair.split("/")
    idx0, idx1 = image_names.index(name0), image_names.index(name1)
    m, sc = get_matches(matches1_dir, name0, name1)
    match_matrix1[idx0, idx1] = match_matrix1[idx1, idx0] = m.shape[0]

ax = sns.heatmap(match_matrix1, linewidth=0.0, cmap="hot", mask=match_matrix1 < 0)

In [ ]:
pairs = sorted(list_h5_names(matches2_dir))
match_matrix2 = -np.ones([len(image_names), len(image_names)])
for pair in pairs:
    name0, name1 = pair.split("/")
    idx0, idx1 = image_names.index(name0), image_names.index(name1)
    m, sc = get_matches(matches2_dir, name0, name1)
    match_matrix2[idx0, idx1] = match_matrix2[idx1, idx0] = m.shape[0]

ax = sns.heatmap(match_matrix2, linewidth=0.0, cmap="hot", mask=match_matrix2 < 0)

In [ ]:
pairs = sorted(list_h5_names(matches1_dir))
match_matrix_ens = -np.ones([len(image_names), len(image_names)])
for pair in pairs:
    name0, name1 = pair.split("/")
    idx0, idx1 = image_names.index(name0), image_names.index(name1)
    m, sc = get_matches(ens_matches_path, name0, name1)
    match_matrix_ens[idx0, idx1] = match_matrix_ens[idx1, idx0] = m.shape[0]

ax = sns.heatmap(match_matrix_ens, linewidth=0.0, cmap="hot", mask=match_matrix_ens < 0)

In [ ]:
name1 = image_names[7]
name2 = image_names[23]

plot_images([read_image(images / name1), read_image(images / name2)])
kp0, kp1 = get_keypoints(features1_dir, name1), get_keypoints(features1_dir, name2)
m, sc = get_matches(matches1_dir, name1, name2)

plot_keypoints([kp0, kp1])
plot_matches(kp0[m[:,0]], kp1[m[:,1]])
add_text(0, f"Matches: {m.shape[0]}")

In [ ]:
plot_images([read_image(images / name1), read_image(images / name2)])
kp0, kp1 = get_keypoints(features2_dir, name1), get_keypoints(features2_dir, name2)
m, sc = get_matches(matches2_dir, name1, name2)

plot_keypoints([kp0, kp1])
plot_matches(kp0[m[:,0]], kp1[m[:,1]])
add_text(0, f"Matches: {m.shape[0]}")

In [ ]:
plot_images([read_image(images / name1), read_image(images / name2)])
kp0, kp1 = get_keypoints(ens_feat_path, name1), get_keypoints(ens_feat_path, name2)
m, sc = get_matches(ens_matches_path, name1, name2)

plot_keypoints([kp0, kp1])
plot_matches(kp0[m[:,0]], kp1[m[:,1]])
add_text(0, f"Matches: {m.shape[0]}")

# SfM

In [ ]:
import shutil

pairs_path = Path(f"../outputs/{NAME1}/{dataset}/{scene}/pairs.txt")

shutil.copy(pairs_path, ens_matches_path.parent)

In [ ]:
from hloc import reconstruction

sfm_dir = Path(f"../outputs/ensemble/{dataset}/{scene}/sparse")
sfm_dir.mkdir(parents=True, exist_ok=True)

pairs_path = Path(f"../outputs/ensemble/{dataset}/{scene}/pairs.txt")

feature_path = Path(f"../outputs/{NAME1}/{dataset}/{scene}/features.h5")
match_path = Path(f"../outputs/{NAME1}/{dataset}/{scene}/matches.h5")

reconstruction.main(
    sfm_dir=sfm_dir,
    image_dir=images,
    pairs=pairs_path,
    features=feature_path,
    matches=match_path
)

In [ ]:
from hloc import reconstruction

sfm_dir = Path(f"../outputs/ensemble/{dataset}/{scene}/sparse")
sfm_dir.mkdir(parents=True, exist_ok=True)

pairs_path = Path(f"../outputs/ensemble/{dataset}/{scene}/pairs.txt")

feature_path = Path(f"../outputs/{NAME2}/{dataset}/{scene}/features.h5")
match_path = Path(f"../outputs/{NAME2}/{dataset}/{scene}/matches.h5")

reconstruction.main(
    sfm_dir=sfm_dir,
    image_dir=images,
    pairs=pairs_path,
    features=feature_path,
    matches=match_path
)

In [ ]:
from hloc import reconstruction

sfm_dir = Path(f"../outputs/ensemble/{dataset}/{scene}/sparse")
sfm_dir.mkdir(parents=True, exist_ok=True)

pairs_path = Path(f"../outputs/ensemble/{dataset}/{scene}/pairs.txt")

feature_path = Path(f"../outputs/ensemble/{dataset}/{scene}/ens_features.h5")
match_path = Path(f"../outputs/ensemble/{dataset}/{scene}/ens_matches.h5")

reconstruction.main(
    sfm_dir=sfm_dir,
    image_dir=images,
    pairs=pairs_path,
    features=feature_path,
    matches=match_path
)

In [ ]:
shutil.rmtree("../outputs/ensemble")